In [3]:
class ContextManger:
    def __init__(self):
        self.entered = False
    
    def __enter__(self):
        self.entered = True
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.entered = False
    
        
cm = ContextManger()
cm.entered

False

In [4]:
with cm:
    print(cm.entered)

True


In [5]:
cm.entered

False

In [6]:
with ContextManger() as a:
    print(a)

None


### 2.32 避免重复


In [7]:
class BubbleException:
    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_val:
            print("bubbling up exception:%s." % exc_val)
        return False
    
with BubbleException():
    print(5 + 5)

10


In [8]:
with BubbleException():
    print(5 / 0)

bubbling up exception:division by zero.


ZeroDivisionError: division by zero

In [9]:
class SuppressException:
    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_val:
            print("suppressing exception:%s." % exc_val)
        return True
    
with SuppressException():
    print(5/0)

suppressing exception:division by zero.


In [10]:
class HandleValueError:
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if not exc_type:
            return True
        if isinstance(exc_val, ValueError):
            print("handling valueerror:%s" % exc_val)
            return True
        return False

with HandleValueError():
    raise ValueError("wrong value.")

handling valueerror:wrong value.


In [11]:
with HandleValueError():
    raise TypeError("wrong value.")

TypeError: wrong value.

### 不包括的子类

In [15]:
class ValueErrorSubclass(ValueError):
    pass


class HandleValueError:
    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        if not exc_val:
            return True
        if xc_val == ValueError:
            print("handling valueError:%s" % exc_val)
            return True
        return False

with HandleValueError():
    raise ValueErrorSubclass("hhh")

handling valueError:hhh


In [17]:
with HandleValueError():
    raise ValueError("hhh")

handling valueError:hhh


基于属性的异常处理

In [18]:
import subprocess


class ShellException(Exception):
    def __init__(self, code, stdout='', stderr=''):
        self.code = code
        self.stdout = stdout
        self.stderr = stderr
        
    def __str__(self):
        return "exit code %d - %d" %(self.code, self.stderr)
    
    
def run_command(command):
    proc = subprocess.Popen(command.split(' '), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    proc.wait()
    stdout, stderr = proc.communicate()
    if proc.returncode > 0:
        raise ShellException(proc.returncode, stdout=stdout, stderr=stderr)
    return stdout

run_command("rm bogus")

FileNotFoundError: [WinError 2] 系统找不到指定的文件。

In [31]:
class AcceptableErrorCodes(object):
    def __init__(self, *error_codes):
        self.error_codes = error_codes
        
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        # print(exc_val,exc_type,vars(exc_val))
        if not exc_type:
            return True
        if not issubclass(exc_type, ShellException):
            return False
        return exc_val.code in self.error_codes

In [32]:
with AcceptableErrorCodes(1):
    run_command("rm aaaa")
    
# win好像这个例子不太合适

[WinError 2] 系统找不到指定的文件。 <class 'FileNotFoundError'> {}


FileNotFoundError: [WinError 2] 系统找不到指定的文件。

## 2.4 更简单的语法

In [34]:
import contextlib


@contextlib.contextmanager
def acceptable_error_codes(*codes):
    try:
        yield 
    except ShellException as exc_val:
        if exc_val.code not in codes:
            raise 
        pass

In [35]:
with acceptable_error_codes(1):
    run_command("rm bbbb")

FileNotFoundError: [WinError 2] 系统找不到指定的文件。